In [89]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline 

In [2]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-17-2020.csv')

In [3]:
latest_data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-17 23:30:52,34.223334,-82.461707,10,0,0,10,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-17 23:30:52,30.295065,-92.414197,110,6,0,104,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-17 23:30:52,37.767072,-75.632346,28,0,0,28,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-17 23:30:52,43.452658,-116.241552,576,9,0,567,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-17 23:30:52,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"


In [4]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,714,784,840,906,933,996,1026,1092,1176,1279
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,475,494,518,539,548,562,584,609,634,663
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,659,673,673,696,704,713,717,717,723,723
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,19,19,24,24,24,24,25,25


In [6]:

country_list=confirmed_df.iloc[:,1:2].values

country_list=np.unique(country_list)


In [7]:
cols = confirmed_df.keys()

In [8]:
confirmed = confirmed_df.loc[:, cols[4]:cols[-1]]
deaths = deaths_df.loc[:, cols[4]:cols[-1]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-1]]

In [91]:
def get_data(country):
    days_in_future = 10
    future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
    adjusted_dates = future_forcast[:-10]
    adjusted_dates = adjusted_dates.reshape(1, -1)[0]
    
    cases=[]
    daily_increase=[]
    deaths=[]
    dates = confirmed.keys()
    recoveries=[]
    for i in dates:

        # case studies 
        cases.append(confirmed_df[confirmed_df['Country/Region']==country][i].sum())
        deaths.append(deaths_df[deaths_df['Country/Region']==country][i].sum())
        recoveries.append(recoveries_df[recoveries_df['Country/Region']==country][i].sum())
        
    daily_increase = daily_increase(cases)
        
    def daily_increase(data):
        d = [] 
        for i in range(len(data)):
            if i == 0:
                d.append(data[0])
            else:
                d.append(data[i]-data[i-1])
        return d         
    
#-----------------------------------------------------------
dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
recovery_rate = [] 
total_recovered = [] 
total_active = [] 
china_cases = [] 
china_deaths = [] 
china_recoveries = []  

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # confirmed, deaths, recovered, and active
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)

    # case studies 

    china_cases.append(confirmed_df[confirmed_df['Country/Region']=='China'][i].sum())    
    china_deaths.append(deaths_df[deaths_df['Country/Region']=='China'][i].sum())
    china_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='China'][i].sum())
    

In [71]:
def daily_increase(data):
    d = [] 
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d 
# confirmed cases
world_daily_increase = daily_increase(world_cases)

# deaths
world_daily_death = daily_increase(total_deaths)

# recoveries
world_daily_recovery = daily_increase(total_recovered)


In [72]:
days_since_1_22 = np.array([i for i in range(len(dates))]).reshape(-1, 1)
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)
total_recovered = np.array(total_recovered).reshape(-1, 1)

In [73]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

In [74]:
adjusted_dates = adjusted_dates.reshape(1, -1)[0]

In [75]:
country_list=confirmed_df.iloc[:,1:2].values
country_list=np.unique(country_list)
country_lst = country_list.tolist()
#print(country_list)

In [76]:
#sorting and making data ready for pie chart
unique_countries =  list(latest_data['Country_Region'].unique())
country_confirmed_cases = []
country_death_cases = [] 
country_active_cases = []
country_recovery_cases = []
country_mortality_rate = [] 

no_cases = []
for i in unique_countries:
    cases = latest_data[latest_data['Country_Region']==i]['Confirmed'].sum()
    if cases > 0:
        country_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
        
for i in no_cases:
    unique_countries.remove(i)
    
# sort countries by the number of confirmed cases
unique_countries = [k for k, v in sorted(zip(unique_countries, country_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
for i in range(len(unique_countries)):
    country_confirmed_cases[i] = latest_data[latest_data['Country_Region']==unique_countries[i]]['Confirmed'].sum()
    country_death_cases.append(latest_data[latest_data['Country_Region']==unique_countries[i]]['Deaths'].sum())
    country_recovery_cases.append(latest_data[latest_data['Country_Region']==unique_countries[i]]['Recovered'].sum())
    country_active_cases.append(country_confirmed_cases[i] - country_death_cases[i] - country_recovery_cases[i])
    country_mortality_rate.append(country_death_cases[i]/country_confirmed_cases[i])

    
    
visual_unique_countries = [] 
visual_confirmed_cases = []
others = np.sum(country_confirmed_cases[10:])

for i in range(len(country_confirmed_cases[:10])):
    visual_unique_countries.append(unique_countries[i])
    visual_confirmed_cases.append(country_confirmed_cases[i])
    
visual_unique_countries.append('Others')
visual_confirmed_cases.append(others)

visual_unique_provinces = [] 
visual_confirmed_cases2 = []
#others = np.sum(province_confirmed_cases[10:])
#for i in range(len(province_confirmed_cases[:10])):
#    visual_unique_provinces.append(unique_provinces[i])
#    visual_confirmed_cases2.append(province_confirmed_cases[i])

visual_unique_provinces.append('Others')
visual_confirmed_cases2.append(others)

In [77]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib.gridspec import GridSpec
from tkinter import ALL


class COVIDGraphApp(tk.Tk):
    
    def __init__(self,*arg, **kwargs):
        tk.Tk.__init__(self,*arg, **kwargs)
        container=tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0,weight=1)
        container.grid_columnconfigure(0,weight=1)
        self.frames={}
        
        for F in (StartPage, PageOne, PageTwo, PageThree,PageFour,PageFive):
            
            frame=F(container,self)

            self.frames[F]=frame

            frame.grid(row=0,column=0,sticky="nsew")
            
    
        self.show_frame(StartPage)
    
    def show_frame(self,cont):
        frame=self.frames[cont]
        frame.tkraise() 
        
    def show_frame2(self,cont,f):
        f.grid_remove()        
        frame=self.frames[cont]
        frame.tkraise()         



In [94]:
class StartPage(tk.Frame):
    
    def __init__(self,parent, controller):
            tk.Frame.__init__(self, parent)
            self.controller = controller           
            
            button22 = ttk.Button(self, text="Show World Graphs Over Time",
                        command=lambda: self.go_to_page_two())
            button22.pack()
            
            button33 = ttk.Button(self, text="Show World Daily Numbers Graphs ",
                        command=lambda: self.go_to_page_three())
            button33.pack()
            button44 = ttk.Button(self, text="Show World Log Graphs Over Time",
                        command=lambda: self.go_to_page_four())
            button44.pack()
            
            button55 = ttk.Button(self, text="Show World Top 10 Pie Chart",
                        command=lambda: self.go_to_page_five())
            button55.pack()
            
            self.label1= tk.Label(self, text="Please choose a country from the list:")
            self.label1.pack()
             #----------------------combo box------------------------
            country_lst = country_list.tolist()
            my_str_var = tk.StringVar()
            self.my_combobox = ttk.Combobox(self, textvariable = my_str_var,values=country_lst)
            self.my_combobox.pack(side=tk.TOP)
            button11 = ttk.Button(self, text="Show Country Graphs",
                        command=lambda: self.go_to_page_one())
            button11.pack()
            
    def go_to_page_one(self):
        self.controller.myVar3=self.my_combobox.get()
        self.controller.frames[PageOne].show_graphs()
        self.controller.show_frame(PageOne)
    def go_to_page_two(self):
       
        self.controller.frames[PageTwo].show_graphs()
        self.controller.show_frame(PageTwo)
    def go_to_page_three(self):
      
        self.controller.frames[PageThree].show_graphs()
        self.controller.show_frame(PageThree)
    def go_to_page_four(self):
       
        self.controller.frames[PageFour].show_graphs()
        self.controller.show_frame(PageFour)
    def go_to_page_five(self):
       
        self.controller.frames[PageFive].show_graphs()
        self.controller.show_frame(PageFive)    
#-------------------------------------------------------------------------  

In [95]:
class PageOne(tk.Frame):
    def __init__ (self, parent, controller):
        
        self.controller = controller
        tk.Frame.__init__(self, parent)
        self.label1= tk.Label(self, text="This is country")
        self.label1.pack()
        
     
        button123=ttk.Button(self,text="back to home",
                              command=lambda:controller.show_frame(StartPage))
                            
        button123.pack()
        def clear():
              
                PageOne.grid_remove(self)              
     
        
    def show_graphs(self):

            country=self.controller.myVar3
            
            self.label1.config(text=self.controller.myVar3)           
            
#----------------getting data for the country-----------------------------
            def get_data(country):
                days_in_future = 10
                dates = confirmed.keys()
                cases=[]
                daily_increase=[]
                deaths=[]
                recoveries=[]
                future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
                adjusted_dates = future_forcast[:-10]
                adjusted_dates = adjusted_dates.reshape(1, -1)[0]                
                for i in dates:
                    cases.append(confirmed_df[confirmed_df['Country/Region']==country][i].sum())
                    deaths.append(deaths_df[deaths_df['Country/Region']==country][i].sum())
                    recoveries.append(recoveries_df[recoveries_df['Country/Region']==country][i].sum())

                def daily_increase(data):
                    d = [] 
                    for i in range(len(data)):
                        if i == 0:
                            d.append(data[0])
                        else:
                            d.append(data[i]-data[i-1])
                    return d  
                 
                return cases,deaths,recoveries;
#-----------------------------------------------------------------
            get_data(country)
            cases,deaths,recoveries = get_data(country)
            f=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('{} Confirmed Cases'.format(country), size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            plt.yticks(size=10,rotation=45)
               
            f2=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('{} Death Cases'.format(country), size=10)
            
            plt.title('{} Recovered Cases'.format(country), size=10)
            a=f.add_subplot(111)
            a2=f2.add_subplot(111)
            
            canvas=FigureCanvasTkAgg(f, self)
            canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            canvas2=FigureCanvasTkAgg(f2, self)
            canvas2.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            
            #canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            def country_plot( y1, y3, y4, country):
                
                x=adjusted_dates
                
               
                a.plot(x,y1)
                canvas.draw()
                plt.xlabel('Days Since 1/22/2020', size=10)
                plt.ylabel('Number of Cases', size=10)
                plt.xticks(size=10)
                
               
                #----------------------------------------
               
                a2.plot(x,y3)
                canvas2.draw()   
                plt.xlabel('Days Since 1/22/2020', size=10)
                plt.ylabel('Number of Cases', size=10)
                plt.xticks(size=10)
                plt.yticks(size=10)
               
                  #--------------------------
                  
                f3=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
                a3=f3.add_subplot(111)
                a3.plot(x,y4)
                plt.xlabel('Days Since 1/22/2020', size=10)
                plt.ylabel('Number of Cases', size=10)
                plt.xticks(size=10)
                plt.yticks(size=10,rotation=45)
                canvas3=FigureCanvasTkAgg(f3, self)
                canvas3.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
                canvas3.draw()   

                return canvas
                
            canvas=country_plot(cases, deaths, recoveries, country)



In [80]:
class PageTwo(tk.Frame):
    def __init__ (self, parent, controller):        
        self.controller = controller
        tk.Frame.__init__(self, parent)
        self.label1= tk.Label(self, text="The World Trends")
        self.label1.pack()        
        button23=ttk.Button(self,text="back to home",
                              command=lambda:controller.show_frame(StartPage))
        button23.pack()  
        
    def show_graphs(self):    
            f=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('Coronavirus Cases Over Time', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10) 
            a=f.add_subplot(111)
            canvas=FigureCanvasTkAgg(f, self)
            canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a.plot(adjusted_dates, world_cases)
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            canvas.draw()            
#-------------------------------------------------------------
            f2=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.plot(adjusted_dates, total_deaths)
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            plt.title('Number of Coronavirus Deaths Over Time', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            a2=f2.add_subplot(111)
            canvas2=FigureCanvasTkAgg(f2, self)
            canvas2.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a2.plot(adjusted_dates, total_deaths)
            canvas2.draw()
#----------------------------------------------------------------------
            f3=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.plot(adjusted_dates, total_recovered)
            plt.title('Number of Coronavirus Recoveries Over Time', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            a3=f3.add_subplot(111)
            canvas3=FigureCanvasTkAgg(f3, self)
            canvas3.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a3.plot(adjusted_dates, total_recovered)
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            canvas3.draw()

In [81]:
class PageThree(tk.Frame):
    def __init__ (self, parent, controller):
        
        self.controller = controller
        tk.Frame.__init__(self, parent)
        self.label1= tk.Label(self, text="The World Trends")
        self.label1.pack()        
        button23=ttk.Button(self,text="back to home",
                              command=lambda:controller.show_frame(StartPage))
        button23.pack()  
        
    def show_graphs(self):      
            f=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('World Daily Increases in Confirmed Cases', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10) 
            a=f.add_subplot(111)
            canvas=FigureCanvasTkAgg(f, self)
            canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a.bar(adjusted_dates, world_daily_increase)
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            canvas.draw()
#-------------------------------------------------------------
            f2=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            plt.title('World Daily Increases in Confirmed Deaths', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            a2=f2.add_subplot(111)
            canvas2=FigureCanvasTkAgg(f2, self)
            canvas2.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a2.bar(adjusted_dates, world_daily_death)
            canvas2.draw()
#----------------------------------------------------------------------
            f3=plt.figure(figsize=(5, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('World Daily Increases in Confirmed Recoveries ', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            a3=f3.add_subplot(111)
            canvas3=FigureCanvasTkAgg(f3, self)
            canvas3.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a3.bar(adjusted_dates, world_daily_recovery)
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            canvas3.draw()

In [82]:
class PageFour(tk.Frame):
    def __init__ (self, parent, controller):
        
        self.controller = controller
        tk.Frame.__init__(self, parent)
        self.label1= tk.Label(self, text="The World Log Graphs")
        self.label1.pack()      
        button23=ttk.Button(self,text="back to home",
                              command=lambda:controller.show_frame(StartPage))
        button23.pack()  
        
    def show_graphs(self):    
            f=plt.figure(figsize=(7, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('Log of Number of Coronavirus Cases Over Time', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10) 
            a=f.add_subplot(111)
            canvas=FigureCanvasTkAgg(f, self)
            #I changed the layout of the next plot to go to next line:
            canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
            a.plot(adjusted_dates, np.log10(world_cases))
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            canvas.draw()
            
#-------------------------------------------------------------
            f2=plt.figure(figsize=(6, 4), dpi=80, facecolor='#E0E0E0')
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            plt.title('Log of Number of Coronavirus Deaths Over Time', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            a2=f2.add_subplot(111)
            canvas2=FigureCanvasTkAgg(f2, self)
            canvas2.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a2.plot(adjusted_dates, np.log10(total_deaths))
            canvas2.draw()
#----------------------------------------------------------------------
            f3=plt.figure(figsize=(6, 4), dpi=80, facecolor='#E0E0E0')
            plt.title('Log of Number of Coronavirus Recoveries Over Time', size=10)
            plt.xlabel('Days Since 1/22/2020', size=10)
            plt.ylabel('Number of Cases', size=10)
            a3=f3.add_subplot(111)
            canvas3=FigureCanvasTkAgg(f3, self)
            canvas3.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
            a3.plot(adjusted_dates, np.log10(total_recovered))
            plt.xticks(size=10)
            plt.yticks(size=10,rotation=60)
            canvas3.draw()

In [83]:
class PageFive(tk.Frame):
    def __init__ (self, parent, controller):
        
        self.controller = controller
        tk.Frame.__init__(self, parent)
        self.label1= tk.Label(self, text="The World top 10 Pie Chart")
        self.label1.pack()
        button24=ttk.Button(self,text="back to home",
                              command=lambda:controller.show_frame(StartPage))
        button24.pack()  
        
    def show_graphs(self):
            xp=visual_unique_countries
            yp=visual_confirmed_cases
            title='Covid-19 Confirmed Cases per Country'
            f=plt.figure(figsize=(7, 7), dpi=80, facecolor='#E0E0E0')
            c = random.choices(list(mcolors.CSS4_COLORS.values()),k = len(unique_countries))
            labels=np.unique(xp)
            explode = (0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05)
            plt.title(title, size=10)            
            a=f.add_subplot(111)
            a.clear()
            a.pie(yp, colors=c,explode = explode, autopct='%1.1f%%')
            f.legend(xp, loc='best', fontsize=10)
            canvas=FigureCanvasTkAgg(f, self)
            canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
            canvas.draw()
        
 

In [98]:
app=COVIDGraphApp()
app.mainloop()